In [21]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from config import g_key
import datetime as dt
import pymongo
import requests
import time

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\swats\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [22]:
#url to be scraped

most_engaging_url='https://www.roblox.com/games#/sortName/Curated_96'
popular_url = 'https://www.roblox.com/games#/sortName/Popular'
top_earning='https://www.roblox.com/games#/sortName/TopGrossing'


In [23]:
# Retrieve page with the requests module
browser.visit(most_engaging_url)

In [24]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
time.sleep(60)
soup = BeautifulSoup(html, 'html.parser')

In [25]:
soup.body.text

'GamesAvatar ShopCreateRobuxGamesAvatar ShopCreateRobuxSearch "" in GamesSearch "" in PlayersSearch "" in Avatar ShopSearch "" in GroupsSearch "" in LibrarySign UpLog InPlease enable Javascript to use all the features on this site.Most EngagingSee All Dust: Wasteland Survival63%1.1K[2X LOGIA ❤️VALENTINE EVENT]Grand Piece Online94%12.3KWarrior Cats🌼Ultimate Edition [BETA]95%1.7KDIMENSION 5⛩️ Anime Fighting Simulator95%69.5K(Sweet Retreat Event) Loomian Legacy 92%5.9K[2x EXP] Shindo94%60.9KElectric State DarkRP(Beta)84%1.1K[🚀BOSSES🚀] Super Power Fighting Simulator91%4.2K🔨 Plane Crazy80%7.2KFLOORS | Arcade Empire91%2.6KBee Swarm Simulator94%29.5K[⚡SUPER EVENT⚡] Bubble Gum Simulator88%29.5KWorld // Zero94%8.8K🎮[XBOX + Gems!] Giant Simulator84%4K[🏆UPDATE 13🏆] 🧪Science Simulator!83%2KPRABIKI! Creatures of Sonaria90%5K💥 Super Power Training Simulator72%2.4KTheme Park Tycoon 290%10.3K『HEAVEN』Your Bizarre Adventure 85%16KRise of Nations86%2.5KSwordburst 287%1.1KSCP-3008 [2.4]90%10.7KIslands 🐔 [

In [26]:
print(soup.prettify())

<html>
 <!--<![endif]-->
 <head data-machine-id="WEB1146">
  <style type="text/css">
   [uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.right-

In [27]:
results = soup.find_all("li", class_="game-card game-tile")
print(results)

[]


In [28]:
# Loop through returned results
title_list = []
users_list=[]
rating_list=[]
game_link_list=[]
game_image_list=[]

for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('div', class_="game-card-name game-name-title").text
        num_users=result.find('span', class_="info-label playing-counts-label").text
        rating=result.find('span', class_="info-label vote-percentage-label").text
        game_link=result.a['href']
        game_image=result.img['src']
       
    # Identify and return link to listing
  

        # Print results only if title, price, and link are available
        if (title and num_users and rating and game_link):  #and game_image and game_link
            title_list.append(title)
            users_list.append(num_users)
            rating_list.append(rating)
            game_link_list.append(game_link)
            game_image_list.append(game_image)
            
    except AttributeError as e:
        print(e)
        
roblox_df=pd.DataFrame({'Title': title_list,
            'User Count': users_list,
            'Positive Ratings': rating_list,
            'Game Link URL': game_link_list,
            'Game Image URL': game_image_list})
           
roblox_df


,Title,User Count,Positive Ratings,Game Link URL,Game Image URL


In [16]:
#manipulate %
roblox_df['Positive Ratings'] = list(map(lambda x: x[:-1], roblox_df['Positive Ratings'].values))

#manipulate user count #
roblox_df['User Count'] = (roblox_df['User Count']).str.replace('K','000')
roblox_df['User Count']=(roblox_df['User Count']).str.replace(".","")

#Format numbers
roblox_df['User Count'] = (roblox_df['User Count']).astype('float64')
roblox_df['Positive Ratings'] = (roblox_df['Positive Ratings']).astype('float64')

roblox_df

,Title,User Count,Positive Ratings,Game Link URL,Game Image URL
0,Dust: Wasteland Survival,11000.0,63.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t7.rbxcdn.com/10fa287b35c40c682332f33f...
1,[2X LOGIA ❤️VALENTINE EVENT]Grand Piece Online,122000.0,94.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t2.rbxcdn.com/896688dd962fb6854ad67a59...
2,Warrior Cats🌼Ultimate Edition [BETA],17000.0,95.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t1.rbxcdn.com/03c98ddfd5a5f8b8650c2bf5...
3,DIMENSION 5⛩️ Anime Fighting Simulator,70000.0,95.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t7.rbxcdn.com/e3063a450fe0ef4c0af96535...
4,(Sweet Retreat Event) Loomian Legacy,59000.0,92.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t7.rbxcdn.com/44fd43c5f806da81a9923981...
5,[2x EXP] Shindo,609000.0,94.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t6.rbxcdn.com/592c76a62082b19b86a519f7...
6,Electric State DarkRP(Beta),11000.0,84.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t3.rbxcdn.com/fa4c3456ebfa8a0c624b41be...
7,[🚀BOSSES🚀] Super Power Fighting Simulator,42000.0,91.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t1.rbxcdn.com/2168d39144690bb9916b5fc9...
8,🔨 Plane Crazy,72000.0,80.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t5.rbxcdn.com/333d6434b04da41177e26068...
9,FLOORS | Arcade Empire,26000.0,91.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t2.rbxcdn.com/5cf92a61d8f198749af9ad27...


In [17]:
sort_df=roblox_df.sort_values(by='User Count', ascending=False)


browser.quit()

In [18]:
sort_df

,Title,User Count,Positive Ratings,Game Link URL,Game Image URL
27,Welcome to Bloxburg,1272000.0,92.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t4.rbxcdn.com/f4ea2e19483349646e4d704e...
5,[2x EXP] Shindo,609000.0,94.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t6.rbxcdn.com/592c76a62082b19b86a519f7...
40,[UPDATE 13] Blox Fruits,515000.0,92.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t3.rbxcdn.com/2fe384952ebad40491a5bedf...
38,Build A Boat For Treasure,429000.0,92.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t6.rbxcdn.com/efb0ad2467d0b23da2e1be95...
37,Gacha Online| Custom OC RP⭐ (Roleplay),366000.0,68.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t5.rbxcdn.com/91e0bb74c330a239eef250bd...
10,Bee Swarm Simulator,295000.0,94.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t2.rbxcdn.com/5a7af67e8aff4511fa2d2e8b...
11,[⚡SUPER EVENT⚡] Bubble Gum Simulator,294000.0,88.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t3.rbxcdn.com/7fd8695da03254565f8dc711...
22,Islands 🐔 [CHICKENS!],209000.0,87.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t4.rbxcdn.com/01d19a5d275c8a7f109dd96f...
18,『HEAVEN』Your Bizarre Adventure,165000.0,85.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t7.rbxcdn.com/45491fb6ce39344b2e131f36...
42,My Dragon Tycoon🐲,149000.0,94.0,https://www.roblox.com/games/refer?IsLargeGame...,https://t0.rbxcdn.com/7b173d2064dda7eb3785a7c4...


In [19]:
game_name = sort_df['Title']
new_df = pd.DataFrame(columns = ['Game Name','Videos','Video Thumbnails','Video url','Published on '], index = range(1000))
titles = []
pub_time = []
thumb_nail = []
vid = []
likes = []
views = []
comments = []
vdf = []
name = []


for i in range(25):
    
    keyword = f'learn roblox {game_name[i]}'
    url1 = f"https://youtube.googleapis.com/youtube/v3/search?part=snippet&maxResults=25&q={keyword}&key={g_key}"
    response = requests.get(url1)
    data = response.json()
    titles.append((data['items'][i]['snippet']['title']))
    pub_time.append(data['items'][i]['snippet']['publishTime'].split('T')[0])
    thumb_nail.append(data['items'][i]['snippet']['thumbnails']['default']['url'])
    video_id = data['items'][i]['id']['videoId']
    vid.append('https://www.youtube.com/watch?v=' + data['items'][i]['id']['videoId'])
    url2 = f"https://youtube.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={video_id}&key={g_key}"
    response = requests.get(url2)
    data2 = response.json()
    try:
        likes.append(int(data2['items'][0]['statistics']['likeCount']))
    except:
        likes.append(0)
    try:
        views.append(int(data2['items'][0]['statistics']['viewCount']))
    except:
        views.append(0)
    try:
        comments.append(int(data2['items'][0]['statistics']['commentCount']))
    except:
        comments.append(0)
    name.append(game_name)
    video_df = pd.DataFrame({'Game Name' : name,  'Videos': titles,  'Views':views, 'Likes': likes,'Comments': comments,'Video Thumbnails': thumb_nail, 'Video url': vid, 'Published on ': pub_time })
    video_df.sort_values(by='Views', ascending=False, inplace = True)
    vdf.append(video_df)
  


KeyError: 'items'

In [20]:
vdf

[]

In [23]:
final_df = vdf.merge(sort_df, )

,Videos,Views,Likes,Comments,Video Thumbnails,Video url,Published on
0,ALL 33 NEW *DIMENSION 5* UPDATE CODES! Anime F...,43944,1425,278,https://i.ytimg.com/vi/g3FimKpeKpc/default.jpg,https://www.youtube.com/watch?v=g3FimKpeKpc,2021-03-06
0,ALL 33 NEW *DIMENSION 5* UPDATE CODES! Anime F...,43944,1425,278,https://i.ytimg.com/vi/g3FimKpeKpc/default.jpg,https://www.youtube.com/watch?v=g3FimKpeKpc,2021-03-06
1,These *NEW* Anime Fighting Simulator Codes Are...,137,16,21,https://i.ytimg.com/vi/8DFO68dNCbE/default.jpg,https://www.youtube.com/watch?v=8DFO68dNCbE,2021-03-13
0,ALL 33 NEW *DIMENSION 5* UPDATE CODES! Anime F...,43944,1425,278,https://i.ytimg.com/vi/g3FimKpeKpc/default.jpg,https://www.youtube.com/watch?v=g3FimKpeKpc,2021-03-06
2,EASY BEAST TITAN SPECIAL UNLOCK CODES IN ANIME...,28899,1262,173,https://i.ytimg.com/vi/8adu1HyKkb8/default.jpg,https://www.youtube.com/watch?v=8adu1HyKkb8,2021-03-12
...,...,...,...,...,...,...,...
24,*MARCH 2021* ALL *NEW* WORKING CODES FOR ANIME...,174,17,6,https://i.ytimg.com/vi/PBLFpBNGRww/default.jpg,https://www.youtube.com/watch?v=PBLFpBNGRww,2021-03-12
15,ALL *34* NEW SECRET OP CODES IN ANIME FIGHTING...,156,9,3,https://i.ytimg.com/vi/9xnHnWGoE-c/default.jpg,https://www.youtube.com/watch?v=9xnHnWGoE-c,2021-03-11
1,These *NEW* Anime Fighting Simulator Codes Are...,137,16,21,https://i.ytimg.com/vi/8DFO68dNCbE/default.jpg,https://www.youtube.com/watch?v=8DFO68dNCbE,2021-03-13
20,ALL CODES IN UPDATE (DIMENSION 5!)-ANIME FIGHT...,102,6,3,https://i.ytimg.com/vi/aO_xMl9yiDQ/default.jpg,https://www.youtube.com/watch?v=aO_xMl9yiDQ,2021-03-13


In [49]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.robloxdb
roblox= db.robloxdb.find()
#insert a dataframe into mongo DB ???????

In [50]:
data_dict = vdf.to_dict('records')

In [51]:
db.roblox.insert_many(data_dict)